In [23]:
pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


In [11]:
import re
import nltk
nltk.download('punkt')
import string

ban_wds = ['anal',
 'anus',
 'arse',
 'ass',
 'ballsack',
 'balls',
 'bastard',
 'bitch',
 'biatch',
 'bloody',
 'blowjob',
 'blow job',
 'bollock',
 'bollok',
 'boner',
 'boob',
 'bugger',
 'bum',
 'butt',
 'buttplug',
 'clitoris',
 'cock',
 'coon',
 'crap',
 'cunt',
 'damn',
 'dick',
 'dildo',
 'dyke',
 'fag',
 'feck',
 'fellate',
 'fellatio',
 'felching',
 'fuck',
 'f u c k',
 'fudgepacker',
 'fudge packer',
 'flange',
 'Goddamn',
 'God damn',
 'hell',
 'homo',
 'jerk',
 'jizz',
 'knobend',
 'knob end',
 'labia',
 'lmao',
 'lmfao',
 'muff',
 'nigger',
 'nigga',
 'omg',
 'penis',
 'piss',
 'poop',
 'prick',
 'pube',
 'pussy',
 'queer',
 'scrotum',
 'sex',
 'shit',
 's hit',
 'sh1t',
 'slut',
 'smegma',
 'spunk',
 'tit',
 'tosser',
 'turd',
 'twat',
 'vagina',
 'wank',
 'whore',
 'wtf',
 'sucker',
 'pornhub',
 'xvideo',
 'assfuck',
 'chinkchong',
 'nigger',
 'chinkn',
 'negro',
 'smalleye',
 'porn',
 'pron','nazi']
#Filter function

def word_dect(text):
    text_cln = text.lower()
    text_cln = re.sub('[^A-Za-z0-9]+', ' ', text) #word tokenization and preporcessing
    text_cln = re.sub(r'[^\w\s]', '', text_cln)
    tokens = nltk.word_tokenize(text_cln)
    if len(tokens) > 2: # check if the number of tokens is larger than 2
        return False
    for i in tokens:
        if i in ban_wds:
            return False
            break
        else:
            continue
    return True





[nltk_data] Downloading package punkt to /Users/hanli/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [16]:
word_dect('sucker,leooh,leoohleoohleoohleoohleoohleooh')

False

In [13]:
# Load Mini Bert model and tokenizer
model = AutoModelForSequenceClassification.from_pretrained("mrm8488/bert-tiny-finetuned-sms-spam-detection",
                                                           num_labels=2, output_attentions=True)
tokenizer = AutoTokenizer.from_pretrained("mrm8488/bert-tiny-finetuned-sms-spam-detection")

# Example input sentences
input_sentences = ["I love pizza"]

# Tokenize input sentences
input_tokens = tokenizer.batch_encode_plus(input_sentences, return_tensors='pt', padding=True)

# Run input through Mini Bert model and get attention weights for all layers
output = model(input_tokens['input_ids'], attention_mask=input_tokens['attention_mask'], output_attentions=True)
attention_weights = output.attentions


In [18]:
from transformers import AutoTokenizer,AutoModelForSequenceClassification
import torch
import re
import string
import nltk
from nltk.corpus import stopwords
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
stopwords = set(stopwords.words('english'))

def preprocess(text):
    # Lowercase the text
    text = str(text).lower()

    # Remove special characters and symbols
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    # Remove \n symbol
    text = re.sub(r'\n','',text )
    # Remove multiple spaces
    text = re.sub('\s+', ' ', text)
    tokens = word_tokenize(text)
    
    tokens = [token for token in tokens if token not in stopwords]
    text = ' '.join(tokens)

    return text

def spam_dect(text):
    # Load the saved model state dictionary
    path = 'to_use.pth'
    state_dict = torch.load(path)

    # Instantiate the model class and load the saved state dictionary
    model = AutoModelForSequenceClassification.from_pretrained("mrm8488/bert-tiny-finetuned-sms-spam-detection",num_labels = 2)
    model.load_state_dict(state_dict)

        # Move the model to the device
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    # Tokenize the text
    text = preprocess(text)
    tokenizer = AutoTokenizer.from_pretrained("mrm8488/bert-tiny-finetuned-sms-spam-detection")
    inputs = tokenizer(text, padding=True, truncation=True, return_tensors="pt")
    inputs.to(device)
    input_ids = torch.tensor(tokenizer.encode(text, add_special_tokens=True)).unsqueeze(0)

    # Pass the input through the model to get the logits
    with torch.no_grad():
        outputs = model(**inputs,output_attentions=True)
        logits = outputs.logits
        attentions = outputs.attentions[0]
    avg_attentions = torch.mean(attentions, dim=1)[0]

    # Get the predicted label
    _, predicted_label = torch.max(logits, dim=1)

    # Get the important words
    important_words = []
    for i in torch.topk(avg_attentions, k=3).indices:
        important_words.append(tokenizer.decode(input_ids[0][i].tolist()))

    # Print the percentage of label 1
    label_1_percentage = torch.softmax(logits, dim=1)[0][1].item() * 100 
    return "RESULT: Your message is " + str(round(label_1_percentage,2))+ " percent chance to be a scam. " + str(important_words[:3]) + ' are the word-combinations that contributing the most to the prediction.'

[nltk_data] Downloading package punkt to /Users/hanli/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /Users/hanli/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [20]:
spam_dect("Hey yuki, call me when you got home")

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


"RESULT: Your message is 0.01 percent chance to be a scam. ['home hey yuki', 'hey home call', 'home hey yuki'] are the word-combinations that contributing the most to the prediction."